<a href="https://colab.research.google.com/github/davidlealo/sistema-recomendacion/blob/main/recomendacion_genero_por_artista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Recomendación de género a través de artistas

Sistema que busca identificar un género o grupo de estilos musicales para un usuario a través de una aplicación aque correrá en Google Colab o soluciones similares

### Instalar librerias

In [23]:
!pip install scikit-surprise

### Recuperar datos

In [24]:
import os
import zipfile
import requests
import pandas as pd
from surprise import Dataset, Reader

# URL del dataset
url = "https://files.grouplens.org/datasets/hetrec2011/hetrec2011-lastfm-2k.zip"
zip_file = "hetrec2011-lastfm-2k.zip"
extract_dir = "hetrec2011-lastfm-2k"

# Descargar el dataset
if not os.path.exists(zip_file):
    print("Descargando dataset...")
    response = requests.get(url)
    with open(zip_file, 'wb') as f:
        f.write(response.content)
    print("Descarga completa.")

# Descomprimir el dataset
if not os.path.exists(extract_dir):
    print("Descomprimiendo dataset...")
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)
    print("Descompresión completa.")

### Cargar cada archivo en un dataframe

In [25]:
# user_artists.dat - Relación entre usuarios y artistas
user_artists_path = os.path.join(extract_dir, 'user_artists.dat')
df_user_artists = pd.read_csv(user_artists_path, sep='\t')
print("Datos de user_artists:")
print(df_user_artists.head())

# artists.dat - Información sobre los artistas
artists_path = os.path.join(extract_dir, 'artists.dat')
df_artists = pd.read_csv(artists_path, sep='\t')
print("Datos de artists:")
print(df_artists.head())

# tags.dat - Información sobre las etiquetas
# Cargar el archivo tags.dat usando una codificación alternativa
tags_path = os.path.join(extract_dir, 'tags.dat')
try:
    df_tags = pd.read_csv(tags_path, sep='\t', encoding='latin1')  # Cambiar la codificación a latin1
    print("Datos de tags:")
    print(df_tags.head())
except UnicodeDecodeError as e:
    print(f"Error de decodificación: {e}")


# user_friends.dat - Relación de amistad entre usuarios
user_friends_path = os.path.join(extract_dir, 'user_friends.dat')
df_user_friends = pd.read_csv(user_friends_path, sep='\t')
print("Datos de user_friends:")
print(df_user_friends.head())

# user_taggedartists-timestamps.dat - Etiquetas de los usuarios a los artistas con marcas de tiempo
user_taggedartists_timestamps_path = os.path.join(extract_dir, 'user_taggedartists-timestamps.dat')
df_user_taggedartists_timestamps = pd.read_csv(user_taggedartists_timestamps_path, sep='\t')
print("Datos de user_taggedartists_timestamps:")
print(df_user_taggedartists_timestamps.head())

# user_taggedartists.dat - Etiquetas de los usuarios a los artistas
user_taggedartists_path = os.path.join(extract_dir, 'user_taggedartists.dat')
df_user_taggedartists = pd.read_csv(user_taggedartists_path, sep='\t')
print("Datos de user_taggedartists:")
print(df_user_taggedartists.head())

# Preparar los datos para Surprise (usaremos user_artists.dat para recomendaciones)

reader = Reader(rating_scale=(df_user_artists['weight'].min(), df_user_artists['weight'].max()))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'weight']], reader)

Datos de user_artists:
   userID  artistID  weight
0       2        51   13883
1       2        52   11690
2       2        53   11351
3       2        54   10300
4       2        55    8983
Datos de artists:
   id               name                                         url  \
0   1       MALICE MIZER       http://www.last.fm/music/MALICE+MIZER   
1   2    Diary of Dreams    http://www.last.fm/music/Diary+of+Dreams   
2   3  Carpathian Forest  http://www.last.fm/music/Carpathian+Forest   
3   4       Moi dix Mois       http://www.last.fm/music/Moi+dix+Mois   
4   5        Bella Morte        http://www.last.fm/music/Bella+Morte   

                                          pictureURL  
0    http://userserve-ak.last.fm/serve/252/10808.jpg  
1  http://userserve-ak.last.fm/serve/252/3052066.jpg  
2  http://userserve-ak.last.fm/serve/252/40222717...  
3  http://userserve-ak.last.fm/serve/252/54697835...  
4  http://userserve-ak.last.fm/serve/252/14789013...  
Datos de tags:
   tagID     

In [26]:
from surprise import SVD, KNNBasic
from surprise.model_selection import train_test_split, cross_validate

# Crear un set de entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar SVD para el modelo de recomendación
algo = SVD()

# Entrenar el modelo
algo.fit(trainset)

# Evaluar el modelo
predictions = algo.test(testset)

### Entrenamiento y evaluación utilizando Surprise

In [27]:
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split, cross_validate

# Preparación de los datos para Surprise
reader = Reader(rating_scale=(df_user_artists['weight'].min(), df_user_artists['weight'].max()))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'weight']], reader)

# Dividir el conjunto de datos en entrenamiento y prueba
trainset, testset = train_test_split(data, test_size=0.25)

# Usar el algoritmo SVD para crear el modelo de recomendación
modelo_svd = SVD()

# Entrenar el modelo SVD
modelo_svd.fit(trainset)

# Evaluar el modelo SVD
predictions_svd = modelo_svd.test(testset)

# Mostrar algunas predicciones SVD
print("\nAlgunas predicciones SVD:")
for prediction in predictions_svd[:5]:
    print(prediction)

# Usar el algoritmo KNNBasic para crear el modelo de recomendación
modelo_knn = KNNBasic()

# Entrenar el modelo KNNBasic
modelo_knn.fit(trainset)

# Evaluar el modelo KNNBasic
predictions_knn = modelo_knn.test(testset)

# Mostrar algunas predicciones KNNBasic
print("\nAlgunas predicciones KNNBasic:")
for prediction in predictions_knn[:5]:
    print(prediction)

# Evaluar el rendimiento de ambos modelos usando validación cruzada
print("\nEvaluación usando validación cruzada:")
cross_validate(modelo_svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
cross_validate(modelo_knn, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)


Algunas predicciones SVD:
user: 1974       item: 227        r_ui = 676.00   est = 352698.00   {'was_impossible': False}
user: 1513       item: 10491      r_ui = 170.00   est = 352698.00   {'was_impossible': False}
user: 208        item: 4269       r_ui = 195.00   est = 352698.00   {'was_impossible': False}
user: 1042       item: 328        r_ui = 295.00   est = 352698.00   {'was_impossible': False}
user: 1119       item: 12420      r_ui = 17.00   est = 352698.00   {'was_impossible': False}
Computing the msd similarity matrix...
Done computing similarity matrix.

Algunas predicciones KNNBasic:
user: 1974       item: 227        r_ui = 676.00   est = 422.83   {'actual_k': 40, 'was_impossible': False}
user: 1513       item: 10491      r_ui = 170.00   est = 378.00   {'actual_k': 1, 'was_impossible': False}
user: 208        item: 4269       r_ui = 195.00   est = 729.82   {'was_impossible': True, 'reason': 'Not enough neighbors.'}
user: 1042       item: 328        r_ui = 295.00   est = 283.1

{'test_rmse': array([4466.567735  , 4232.90351745, 4362.12097402, 4782.91682757,
        4235.54531037]),
 'test_mae': array([796.98510858, 787.91410122, 753.70707119, 812.25418551,
        750.07184896]),
 'fit_time': (0.4954235553741455,
  0.5557188987731934,
  0.662914514541626,
  0.527944803237915,
  0.43499255180358887),
 'test_time': (1.8554644584655762,
  2.2106130123138428,
  2.689832925796509,
  1.8961353302001953,
  2.0228426456451416)}

### Predicciones

In [28]:
# Realizar una predicción para un usuario específico y un artista específico usando el modelo SVD
user_id = 2  # ID del usuario
artist_id = 51  # ID del artista

# Hacer la predicción con el modelo SVD
pred_svd = modelo_svd.predict(user_id, artist_id)
print(f"Predicción SVD para el usuario {user_id} y el artista {artist_id}: {pred_svd.est}")

# Hacer la predicción con el modelo KNNBasic
pred_knn = modelo_knn.predict(user_id, artist_id)
print(f"Predicción KNNBasic para el usuario {user_id} y el artista {artist_id}: {pred_knn.est}")

Predicción SVD para el usuario 2 y el artista 51: 352698
Predicción KNNBasic para el usuario 2 y el artista 51: 3640.245146862614


## Aplicación

### Versión 1

In [30]:
import random
from IPython.display import Javascript, display
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, last_liked_artist_id=None, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Si hay un artista que le gustó, filtrar por similitud
    if last_liked_artist_id is not None:
        # Filtrar los artistas que tengan un score más cercano al último artista que le gustó
        similar_artists = [(artist_id, score) for artist_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_artist_id).est) < 0.1]
        predictions = similar_artists if similar_artists else predictions  # Si no hay similares, usar todos

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index, last_liked_artist_id
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0
    last_liked_artist_id = None  # Reiniciar el último artista que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_artists = []  # Lista para almacenar artistas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, last_liked_artist_id, liked_artists, like_count

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        last_liked_artist_id = df_artists[df_artists['name'] == artist_name]['id'].values[0]  # Guardar el último artista que le gustó
        liked_artists.append(artist_name)  # Agregar el artista a la lista de artistas que le gustan
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            print("Te gustan los siguientes artistas:")
            for artist in liked_artists:
                print(artist)
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)


Artista recomendado: The Clarendonians, score: 1.396518653523977


<IPython.core.display.Javascript object>

Has elegido: No me gusta para The Clarendonians
Nuevo artista recomendado: Stephanie, score: 1


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Stephanie
Nuevo artista recomendado: Rush of Fools, score: 1


...

Has elegido: Me gusta para Frost
No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Artista recomendado: Мультfильмы, score: 1.284513593984831


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Мультfильмы
Nuevo artista recomendado: Evanescence, score: 1


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Evanescence
Te gustan los siguientes artistas:
Jeff Williams
Mariottide
Venus Hum
Big Mike
Frost
Evanescence


### Versión 2

In [31]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, last_liked_artist_id=None, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Si hay un artista que le gustó, filtrar por similitud
    if last_liked_artist_id is not None:
        # Filtrar los artistas que tengan un score más cercano al último artista que le gustó
        similar_artists = [(artist_id, score) for artist_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_artist_id).est) < 0.1]
        predictions = similar_artists if similar_artists else predictions  # Si no hay similares, usar todos

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index, last_liked_artist_id
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0
    last_liked_artist_id = None  # Reiniciar el último artista que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_artists = []  # Lista para almacenar artistas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, last_liked_artist_id, liked_artists, like_count

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        last_liked_artist_id = df_artists[df_artists['name'] == artist_name]['id'].values[0]  # Guardar el último artista que le gustó
        liked_artists.append(artist_name)  # Agregar el artista a la lista de artistas que le gustan
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            # Mostrar mensaje final en rojo
            final_message = "<h2 style='color: red;'>Te gustan los siguientes artistas:</h2><ul>"
            final_message += ''.join(f"<li style='color: red;'>{artist}</li>" for artist in liked_artists)
            final_message += "</ul>"
            display(HTML(final_message))
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)


Artista recomendado: AirFare, score: 1.3048191383346852


<IPython.core.display.Javascript object>

Has elegido: Me gusta para AirFare
Artista recomendado: Thomas Godoj, score: 1.2993083535309324


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Thomas Godoj
Artista recomendado: KSU, score: 1.2841511043834346


...

Has elegido: Me gusta para Nina Simone
Artista recomendado: Lokua Kanza, score: 1.278412882756935


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Lokua Kanza
No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.
Nuevo usuario seleccionado. Artista recomendado: Billy Crystal Meth, score: 1.3130230893722252


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Billy Crystal Meth


### Versión 3

#### DataFrame para versión 3

In [32]:
# Crear nuevo df merge df_artists (id) y df_user_taggedartists (artistID)
new_artists_user_taggedartists = pd.merge(df_artists, df_user_taggedartists, left_on='id', right_on='artistID')

# Eliminar variables no útiles para el programa
new_artists_user_taggedartists = new_artists_user_taggedartists.drop(columns=['id', 'pictureURL', 'userID', 'day', 'month', 'year'])

# Eliminar en new_artists_user_taggedartists los artistID repetidos
new_artists_user_taggedartists = new_artists_user_taggedartists.drop_duplicates(subset=['artistID'])

# merge new_artists_user_taggedartists (artistID) con df_tags (tagValue)
new_artists_user_taggedartists = pd.merge(new_artists_user_taggedartists, df_tags, left_on='tagID', right_on='tagID')

In [33]:
import random
from IPython.display import Javascript, display, HTML
from google.colab.output import eval_js
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD

# Asumimos que df_user_artists, df_artists y new_artists_user_taggedartists ya están cargados

# Normalizar 'weight' a una escala de 1 a 5
min_weight = df_user_artists['weight'].min()
max_weight = df_user_artists['weight'].max()

df_user_artists['rating'] = 1 + 4 * (df_user_artists['weight'] - min_weight) / (max_weight - min_weight)

# Convertir el DataFrame en el formato esperado por Surprise
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_user_artists[['userID', 'artistID', 'rating']], reader)

# Entrenar el modelo con Surprise
trainset = data.build_full_trainset()
modelo_svd = SVD()
modelo_svd.fit(trainset)

# Función para obtener recomendaciones basadas en predicciones
def get_top_artists_for_user(user_id, last_liked_artist_id=None, n=5):
    # Obtener todos los artistas que el usuario no ha escuchado
    unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])

    # Predecir ratings para estos artistas usando el modelo de Surprise
    predictions = [(artist_id, modelo_svd.predict(user_id, artist_id).est) for artist_id in unheard_artists]

    # Si hay un artista que le gustó, filtrar por similitud
    if last_liked_artist_id is not None:
        # Filtrar los artistas que tengan un score más cercano al último artista que le gustó
        similar_artists = [(artist_id, score) for artist_id, score in predictions if abs(score - modelo_svd.predict(user_id, last_liked_artist_id).est) < 0.1]
        predictions = similar_artists if similar_artists else predictions  # Si no hay similares, usar todos

    # Ordenar por rating predicho y devolver los top n
    return sorted(predictions, key=lambda x: x[1], reverse=True)[:n]

# Inicializar lista de artistas recomendados para un usuario aleatorio
def inicializar_recomendaciones():
    global user_id, recommended_artists, current_index, last_liked_artist_id
    user_id = random.choice(df_user_artists['userID'].unique())  # Seleccionar un usuario al azar
    recommended_artists = get_top_artists_for_user(user_id)
    current_index = 0
    last_liked_artist_id = None  # Reiniciar el último artista que le gustó

inicializar_recomendaciones()  # Generar la primera recomendación

liked_artists = []  # Lista para almacenar artistas que le gustan
like_count = 0  # Contador de "me gusta"

def create_buttons(artist_name):
    artist_name_escaped = artist_name.replace("'", "\\'").replace('"', '\\"')  # Escapar caracteres
    display(Javascript(f'''
        var div = document.createElement('div');
        var btn1 = document.createElement('button');
        btn1.innerHTML = 'Me gusta: {artist_name_escaped}';
        btn1.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['Me gusta', '{artist_name_escaped}'], {{}});
        }};

        var btn2 = document.createElement('button');
        btn2.innerHTML = 'No me gusta: {artist_name_escaped}';
        btn2.onclick = function() {{
          google.colab.kernel.invokeFunction('notebook.callback', ['No me gusta', '{artist_name_escaped}'], {{}});
        }};

        div.appendChild(btn1);
        div.appendChild(btn2);
        document.body.appendChild(div);
    '''))

def handle_choice(choice, artist_name):
    global user_id, recommended_artists, current_index, last_liked_artist_id, liked_artists, like_count

    print(f'Has elegido: {choice} para {artist_name}')

    if choice == 'Me gusta':
        # Obtener el ID del artista
        artist_id = df_artists[df_artists['name'] == artist_name]['id'].values[0]

        # Intentar obtener el tagValue, si existe
        tag_value = "Desconocido"
        if artist_id in new_artists_user_taggedartists['artistID'].values:
            tag_value = new_artists_user_taggedartists[new_artists_user_taggedartists['artistID'] == artist_id]['tagValue'].values[0]

        # Guardar el último artista que le gustó y su etiqueta
        last_liked_artist_id = artist_id
        liked_artists.append((artist_name, tag_value))  # Guardar artista y etiqueta como una tupla
        like_count += 1  # Incrementar el contador de "me gusta"

        # Verificar si ha alcanzado 6 "me gusta"
        if like_count >= 6:
            # Crear el mensaje final con los artistas y sus etiquetas
            final_message = "<h2 style='color: red;'>Te gustan los siguientes artistas:</h2><ul>"
            final_message += ''.join(f"<li style='color: red;'>{artist[0]} - {artist[1]}</li>" for artist in liked_artists)
            final_message += "</ul>"

            # Contar la frecuencia de los tags para sugerir el estilo preferido
            tag_frequencies = pd.Series([artist[1] for artist in liked_artists if artist[1] != "Desconocido"]).value_counts()
            if not tag_frequencies.empty:
                favorite_tag = tag_frequencies.idxmax()
                final_message += f"<h3 style='color: yellow;'>Tu estilo de música preferido es: {favorite_tag}</h3>"
            else:
                final_message += "<h3>No se pudo determinar un estilo de música preferido</h3>"

            display(HTML(final_message))
            return  # Termina el programa

        current_index += 1
        if current_index < len(recommended_artists):
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
        else:
            print("No hay más artistas en la lista de recomendaciones. Seleccionando un nuevo usuario.")
            inicializar_recomendaciones()
            next_artist_id, next_score = recommended_artists[current_index]
            next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
            print(f"Nuevo usuario seleccionado. Artista recomendado: {next_artist_name}, score: {next_score}")
            create_buttons(next_artist_name)
    else:
        # Seleccionar un nuevo artista si elige 'No me gusta'
        unheard_artists = set(df_artists['id']) - set(df_user_artists[df_user_artists['userID'] == user_id]['artistID'])
        next_artist_id = random.choice(list(unheard_artists))
        next_artist_name = df_artists[df_artists['id'] == next_artist_id]['name'].values[0]
        next_score = modelo_svd.predict(user_id, next_artist_id).est
        print(f"Nuevo artista recomendado: {next_artist_name}, score: {next_score}")
        create_buttons(next_artist_name)

# Registrar la función en Google Colab
from google.colab import output
output.register_callback('notebook.callback', handle_choice)

# Mostrar el primer artista recomendado
first_artist_id, first_score = recommended_artists[current_index]
first_artist_name = df_artists[df_artists['id'] == first_artist_id]['name'].values[0]
print(f"Artista recomendado: {first_artist_name}, score: {first_score}")
create_buttons(first_artist_name)

Artista recomendado: LAREINE, score: 1.3808486882266593


<IPython.core.display.Javascript object>

Has elegido: No me gusta para LAREINE
Nuevo artista recomendado: Xavier Baumaxa, score: 1.1936162842018185


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Xavier Baumaxa
Nuevo artista recomendado: Boban Marković Orkestar, score: 1.0495974677118025


...

Has elegido: No me gusta para Iron Fire
Nuevo artista recomendado: Dert Floyd, score: 1


<IPython.core.display.Javascript object>

Has elegido: No me gusta para Dert Floyd
Nuevo artista recomendado: Tom Jones, score: 1


<IPython.core.display.Javascript object>

Has elegido: Me gusta para Tom Jones
